In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display

# Pathfinder Playtest Encounter Creator

## Overview
This is a python based tool for DMs to design encounters for the Pathfinder Playtest Rulset. The goal is to be able to quickly and efficiently design encounters with minimal input from the user.

## Plan and Implementation
To achieve this goal, my approach is to use stepwise design in an implementation similar to the "rod-cutting" dynamic programming problem where all costs for each "length" are equal. That is, the minimum viable product goal is to naievely find all possible permutations of creatures that match our parameters.

The algorithm should take several parameters. 
- Party Level
- Party Size
- Encounter Severirty
- (Optional) The maximum number of creatures for the encounter.
- (Optional) The minimum number of creatures for the encounter.
- (Optional) An XP Budget

### The Algorithm
The algorithm itself will try to attain a combination of creatures that is equal to or under the cost of the encounter severity OR an XP budget if provided. 
To do this it will use top-down greedy programming by first choosing the largest XP cost creature that fits the budget. Once found it will store that local solution into a database and then recurse finding the next highest costing creature that fits. When a creature is found and there is still a remaning budget left, it will then use that new remaining budget and recurse down again choosing the local maximum until the base case where the remaining budget is 0. 

In [69]:
#__________This section contains all of the background setup__________

# Load the bestiary database
bestiary = pd.read_csv('data/bestiary.csv');
#bestiary

# Set the baseline encounter xp values.
BUDGETS = {'Trivial': [40,10], 'Low': [60,15], 'High': [80,20], 'Severe': [120,30], 'Extreme': [160,40]}

In [71]:
#__________This section contains the Algorithm that powers the encounter builder__________

# This global variable is the master list that contains all of the encounters.
encounter_list = []

# This function does most of the heavy lifting.
# Input:
#     catalogue: The bestiary an all of their associated xp values.
#            xp: The experience budget to dictate how many monsters can fit in a given encounter.
#   party_level: The party's level. Used in finding the xp value of a creature relative to the party's level.
#         max_c: The maxmimum amount of creatures allowed in an encounter.
#         min_c: The minimum amount of creatures allowed in an encounter.
#     encounter: (Optional) This contains the current encounter being worked on. Initially starts as empty but
#                           is filled during recursive calls.
# Returns:
#          None
# This function takes a series of parameters to build an encounter and append it to a global list of possible encounters.
# Using recursion, it finds creatures in the bestiary that contain xp values less than the budget and adds it as a leaf
# node in the encounter tree. Then it attempts to recurse and add a new leaf node if there is more xp budget and there 
# is enough space left in the encounter. If neither of those conditions are met, it publishes the current encounter to the 
# global list an removes the leaf node to try a new one.
def find_creature(catalogue, xp, party_level,max_c,min_c,encounter=[]):
    
    # Base Case
    # The xp budget is 0 therefore no new leaf nodes can be added. Immediately publish the current encounter.
    if xp == 0:
        publish(encounter.copy())
    
    # Otherwise iterate through the whole list of monsters to find appropriate children
    else:
        # Loop through every monster
        for index, row in catalogue.iterrows():
            # Check to see if we can afford the xp cost of the current monster.
            if int(row[str(party_level)]) <= xp:
                encounter.append(row['CREATURE NAME'])       # We can afford it so add it as a leaf node to the encounter tree.
                newBudget = xp - int(row[str(party_level)])  # Calcualte the remaining budget.
                if len(encounter) < max_c:                   # If we still have space in the encounter, recursively call and
                                                             # attempt to find a new monster that we can afford.
                    find_creature(catalogue, newBudget, party_level, max_c, min_c,encounter)
                    
                # Once we have used up the budget we can publish.
                # This works because if we haven't added anything to the encounter then we can publish what we have.
                # If it did add something to the encounter however, that level's recursive call will handle the publishing.
                
                if len(encounter) >= min_c:     # If the encounter meets the minimum monster count we can publish.
                    publish(encounter.copy())
                del encounter[-1]               # After publishing the encounter we want to remove the leaf that was used and
                                                # so that we can try a new one.

                    
# This is a helper function to publish an encounter to the encounter list.
# Input:
#   current_enc: Contains the encounter to be published in list form.
# Returns:
#          None
def publish(current_enc):
    global encounter_list
    if len(current_enc) > 0:     # Check to make sure we aren't publishing an empty list.
        if current_enc not in encounter_list:    # Also make sure that we don't publish something that's already there.
            encounter_list.append(current_enc)
    
# This is the driver function that's called from the main body. 
# Input:
#   party_level: The party's level. Used in finding the xp value of a creature relative to the party's level.
#    party_size: The size of the party. Used in conjunction with the severity to calculate the XP budget.
#      severity: Used with the party size to calculate the XP budget in the event a budget is not provided.
#         max_c: The maxmimum amount of creatures allowed in an encounter.
#         min_c: The minimum amount of creatures allowed in an encounter.
#        budget: A manually input XP budget.
# Returns:
#          None
def build_encounters(party_level,party_size,severity,max_c,min_c,budget):
    global encounter_list
    # First, calcualte total encounter budget.
    if budget > 0:
        XP_Budget = budget
    else:
        if party_size > 4:
            XP_Budget = BUDGETS[severity][0] + (BUDGETS[severity][1] * (party_size - 4))
        elif party_size < 4:
            XP_Budget = BUDGETS[severity][0] - (BUDGETS[severity][1] * (4-party_size))
        else:
            XP_Budget = BUDGETS[severity][0]
    
    # Next, consult the database and get the XP values for each creature based on the specified level.
    xp_costs = bestiary[['CREATURE NAME', str(party_level)]]
    nontrivial_xp_costs = xp_costs[xp_costs[str(party_level)] != '-']
    pruned_xp_costs = nontrivial_xp_costs[nontrivial_xp_costs[str(party_level)] != 'X']
    
    # Now that we have the creatures and how much they cost, build all possible encounters.
    find_creature(pruned_xp_costs,XP_Budget,party_level,max_c,min_c) # This function does most of the heavy lifting
                                                                     # and builds the encounter list into a global variable.
    temp_enc_list = encounter_list.copy() # Copy the encounters list into a temporary value to return
    encounter_list = []                   # Scrub the global encounters list clean so that it can be ready for another
                                          # query.
    return temp_enc_list


#Testing Function
#Build an High threat encounter for a party of 4 level 1 adventurers with a max of five monsters and a minimum of one.
#XP Budget: 80
test = build_encounters(1,4,'High',5,1,0)


#encounter_list
#print('Making sure the Encounter List is now empty.',encounter_list)
#test

In [91]:
#__________This section contains the code that drives the interface__________

button = widgets.Button(description="Build Encounters!")
def on_button_clicked(b):
    print('Working...')
    solution = build_encounters(party_level.value, 
                                party_size.value, 
                                severity.value,
                                creature_count.value[1], 
                                creature_count.value[0],
                                custom_budget.value)
    filename = 'PL-'+str(party_level.value)+'_PS-'+str(party_size.value)+'_SEV-'+severity.value
    filename = filename+'_MIN-'+str(creature_count.value[0])+'_Max-'+str(creature_count.value[1])
    filename = filename+'_CB-'+str(custom_budget.value)+'.csv'
    df = pd.DataFrame(solution)
    df.to_csv(filename, index=False)
    print('Encounters saved to file: ',filename)
    
button.on_click(on_button_clicked)

party_level = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    description='Party Level:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)
party_size = widgets.IntSlider(
    value=4,
    min=0,
    max=10,
    description='Party Size:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

severity = widgets.Dropdown(
    options=['Trivial', 'Low', 'High', 'Severe', 'Extreme'],
    value='Trivial',
    description='Encounter Severity: ',
    disabled=False,
    style = {'description_width': 'initial',}
)

custom_budget = widgets.BoundedIntText(
    value='0',
    min=0,
    max=500,
    step=5,
    description='Manual XP Budget (Optional):',
    disabled=False,
    style = {'description_width': 'initial'}
)

creature_count = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=20,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='40%')
)


display(party_level)
display(party_size)
display(severity)
display(custom_budget)

print('')
print('')
print("Select a minimum and maximum number of creatures to use in the encounter:")
display(creature_count)





display(button)

IntSlider(value=1, description='Party Level:', layout=Layout(width='40%'), max=20, min=1, style=SliderStyle(de…

IntSlider(value=4, description='Party Size:', layout=Layout(width='40%'), max=10, style=SliderStyle(descriptio…

Dropdown(description='Encounter Severity: ', options=('Trivial', 'Low', 'High', 'Severe', 'Extreme'), style=De…

BoundedIntText(value=0, description='Manual XP Budget (Optional):', max=500, step=5, style=DescriptionStyle(de…



Select a minimum and maximum number of creatures to use in the encounter:


IntRangeSlider(value=(1, 20), continuous_update=False, layout=Layout(width='40%'), max=20, min=1)

Button(description='Build Encounters!', style=ButtonStyle())

Working...
Encounters saved to file:  PL-1_PS-4_SEV-High_MIN-1_Max-3_CB-0.csv
